In [1]:
import json
class Neo4jGraph():
    def __init__(self, login):
        self.graph = None
        self._ip = login.split('-')[0]
        self._username = login.split('-')[1]
        self._password = login.split('-')[2]
        self.CQL = ''
    def con(self):
        from py2neo import Graph, Node, Relationship, NodeMatcher
        self.graph = Graph(self._ip, username=self._username, password=self._password)
        return self.graph
    def cql(self, string):
        return self.graph.run(string).data()
function_blue_colorflat=1
def blue(string):
    global function_blue_colorflat
    if function_blue_colorflat==0:
        # function_blue_colorflat=1
        return ('\033[0;35;48m'+string+'\033[0m')
    else:
        # function_blue_colorflat = 0
        return ('\033[0;34;48m' + string + '\033[0m')
connection = "http://localhost:7474-neo4j-123456"
graph = Neo4jGraph(connection)
graph.con()
def getNode(Parameters):
    cql = "match (n:Node{NodeId:'"+Parameters+"'})  return n"
    res=graph.cql(cql)
    return res
def getdescribe(Parameters):
    cql = "match (n:Node{NodeId:'"+Parameters+"'}) where n.describe is not null return n"
    res=graph.cql(cql)
    return res
def getWords(centence):
    wordlist=[]
    p=-1
    while p<len(centence):
        p+=1
        if centence[p:p+1] in ['的','是','和','：','，','。','（','）'] :
#             print('222')
            continue
        if p+10<=len(centence):
            for q in range(p+10, p, -1):
                node = getNode(centence[p:q])
                if len(node) != 0:
                    if len(centence[p:q])>=2:
                        wordlist.append(centence[p:q])
                    p = q
                    p -= 1
        else:
            for q in range(len(centence), p, -1):
                node = getNode(centence[p:q])
                if len(node) != 0:
                    if len(centence[p:q]) >= 2:
                        wordlist.append(centence[p:q])
                    p = q
                    p -= 1
    return wordlist
def priblue(centence,wordlist):
    centence1=centence
    centence2=''
    for i in wordlist:
        centence2+=i
    p=0
    q=0
    while p<len(centence1):
        if centence1[p:p+1]==centence2[q:q+1]:
            print(blue(centence1[p:p+1]),end='')
            p+=1
            q+=1
        else:
            print((centence1[p:p+1]),end='')
            p=p+1
    print('')
def lable_count(wordlist):
    lable_dic={}
    for node in wordlist:
        lablelist=graph.cql("match (n:Node{NodeId:'"+node+"'}) return labels(n)")[0]['labels(n)']
        for lable in lablelist:
#             print(lable)
            count=lable_dic.get(lable, 0)
            lable_dic[lable]=count+1
    for i in ['Node']:
        if i in lable_dic:
            lable_dic.pop(i)
    return sorted(lable_dic.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)
def Recommender_system(lablelist,n):
    
    for count in range(len(lablelist)-1,-1,-1):
        lables=''
        for i in range(count):
            lables+=':'+lablelist[i][0]
        cql='match (n'+lables+') return n.NodeId limit '+str(n)
        nodes=graph.cql(cql)
        if len(nodes)!=0:
            return nodes
def recog(string):
    wordlist=getWords(string)
    print(string,end='')
    print('='*50+'List'+'='*50+'\n',wordlist)
    lable_list=lable_count(wordlist)
    print('='*50+'Lable'+'='*50+'\n',lable_list)
    recommender_system=Recommender_system(lable_list,10)
    print('='*50+'RecommenderSystem'+'='*50)
    for i in recommender_system:
        print(i)
def Nodelist(string):
    wordlist=getWords(string)
    priblue(string,wordlist)
#     print(string,end='')
    print('='*50+'List'+'='*50+'\n',wordlist)
def Nodelable(string):
    wordlist=getWords(string)
    lable_list=lable_count(wordlist)
    print('='*50+'Lable'+'='*50)
    for i in lable_list:
        print(i)
def RecommendSys(string):
    wordlist=getWords(string)
    lable_list=lable_count(wordlist)
    recommender_system=Recommender_system(lable_list,10)
    print('='*50+'RecommenderSystem'+'='*50)
    for i in recommender_system:
        print(i)
        
# 推理
def fanyi(string):
    from 有道API import youdaoAPI
    ss=youdaoAPI.youdaofanyi(string)
    t = ss.split(' ')
    for i in range(len(t)):
        if i != 0:
            t[i] = t[i].capitalize()
    s = ''
    for i in t:
        s += i
    return s

def addWords(string):
    graph.cql("merge (n:Node{NodeId:'"+string+"'}) return n")
def logic(string):
#     print('答：')
    s=['是','有','什么','哪些','谁','多少']
    trans_en=[]
    for i in s:
        string=string.replace(i,'')
    wordlist=getWords(string)
#     print(wordlist)
    for i in wordlist:
#         print()
        trans_en.append(fanyi(i))
    if len(wordlist)==1:
        describe=getdescribe(wordlist[0])
        if len(describe)!=0:
            line=str(describe).split("describe='")
            print(line[1][:line[1].find("'")])
    if len(wordlist)>=2:
        res=graph.cql("match (a:Node{NodeId:'"+wordlist[0]+"'})-[c:"+trans_en[1]+"]->(b) return b")
        list_ans=[]
        for i in res:
            line=str(i)
            one=line[line.find("NodeId='")+len("NodeId='"):line.find("'",line.find("NodeId='")+len("NodeId='"))]
            if one not in list_ans:
                list_ans.append(one)
#         print('A:',end='')
        for i in list_ans:
            print(i)
#         print(list_ans)  
# 属性
def property(NodeId):
    res=graph.cql("match (a:Node{NodeId:'"+NodeId+"'})-[c]->(b:Node) return c")
    pros=[]
    for i in res:
        line=str(i).split("NodeId")
        pros.append([line[0][line[0].find("'c': ")+len("'c': "):line[0].find("(Node")],line[2][line[2].find("='")+len("='"):line[2].find("'",2)]])
    
    for i in pros:
        print(' : '.join(i))
#     print(pros)
def QA_test():
    question=input(' '*40+'Q:')
    while question!='bye':
        print('-'*100+'\nA: ',end='')
        logic(question)
        question=input(' '*40+'Q:')
    print('Bye!')
        


# 搜索之实体识别

### Part 1

In [2]:
# Entity recognition
string1='''
Facebook（脸书）公司创立于2004年2月4日，总部位于美国加利福尼亚州门洛帕克。
'''

In [3]:
Nodelist(string1)


Facebook（脸书）公司创立于2004年2月4日，总部位于美国加利福尼亚州门洛帕克。

==================================================List==================================================
 ['Facebook', '脸书', '公司', '创立于2004年', '2月4日', '总部', '位于美国加利福尼亚州', '门洛帕克']


In [4]:
Nodelable(string1)

==================================================Lable==================================================
('组织机构', 2)
('公司', 2)
('节日', 1)
('网站', 1)
('法律术语', 1)
('外国行政区划', 1)
('地理', 1)
('地点', 1)
('历史年代', 1)
('历史', 1)


In [5]:
RecommendSys(string1)

==================================================RecommenderSystem==================================================
{'n.NodeId': '株式会社'}
{'n.NodeId': '上市公司'}
{'n.NodeId': '7-11日本伊藤洋华堂公司所属便利店'}
{'n.NodeId': '联想集团'}
{'n.NodeId': '微软'}
{'n.NodeId': '宝马汽车品牌'}
{'n.NodeId': '河北巨龙药业有限责任公司'}
{'n.NodeId': '石家庄众美广告'}
{'n.NodeId': '空中客车公司'}
{'n.NodeId': '波音美国航空航天制造公司'}


###  Part 2

In [6]:
# Entity recognition
string2='''
《再别康桥》是现代诗人徐志摩脍炙人口的诗篇，是新月派诗歌的代表作品。
'''

In [7]:
Nodelist(string2)


《再别康桥》是现代诗人徐志摩脍炙人口的诗篇，是新月派诗歌的代表作品。

==================================================List==================================================
 ['《再别康桥》', '现代诗人', '徐志摩', '脍炙人口', '诗篇', '新月派诗歌', '代表作品']


In [8]:
Nodelable(string2)

==================================================Lable==================================================
('书籍', 2)
('诗词', 1)
('文学作品', 1)
('文化', 1)
('成语', 1)
('字词', 1)
('出版物', 1)
('中国文学', 1)


In [9]:
RecommendSys(string2)

==================================================RecommenderSystem==================================================
{'n.NodeId': '斋居春久感事遣怀'}
{'n.NodeId': '不与老为期'}
{'n.NodeId': '得潮州杨相公继之书并诗以此寄之'}
{'n.NodeId': '贺新郎词牌名'}
{'n.NodeId': '一骑红尘妃子笑'}
{'n.NodeId': '箜篌引诗歌作品'}
{'n.NodeId': '隐括词'}
{'n.NodeId': '古风·大雅久不作'}
{'n.NodeId': '应天长'}
{'n.NodeId': '喝火令'}


# 识别段落中的实体

In [2]:
Nodelist(input("P:"))

P:Facebook（脸书，脸谱网）公司创立于2004年2月4日，总部位于美国加利福尼亚州门洛帕克。
Facebook（脸书，脸谱网）公司创立于2004年2月4日，总部位于美国加利福尼亚州门洛帕克。
==================================================List==================================================
 ['Facebook', '脸书', '脸谱', '公司', '创立于2004年', '2月4日', '总部', '位于美国加利福尼亚州', '门洛帕克']


# 段落标签排序

In [3]:
Nodelable(input("P:"))

P:Facebook（脸书，脸谱网）公司创立于2004年2月4日，总部位于美国加利福尼亚州门洛帕克。
==================================================Lable==================================================
('组织机构', 2)
('公司', 2)
('节日', 1)
('网站', 1)
('法律术语', 1)
('外国行政区划', 1)
('地理', 1)
('地点', 1)
('历史年代', 1)
('历史', 1)


# 基于标签的实体推荐

In [4]:
RecommendSys(input("P:"))

P:Facebook（脸书，脸谱网）公司创立于2004年2月4日，总部位于美国加利福尼亚州门洛帕克。
==================================================RecommenderSystem==================================================
{'n.NodeId': '株式会社'}
{'n.NodeId': '上市公司'}
{'n.NodeId': '7-11日本伊藤洋华堂公司所属便利店'}
{'n.NodeId': '联想集团'}
{'n.NodeId': '微软'}
{'n.NodeId': '宝马汽车品牌'}
{'n.NodeId': '河北巨龙药业有限责任公司'}
{'n.NodeId': '石家庄众美广告'}
{'n.NodeId': '空中客车公司'}
{'n.NodeId': '波音美国航空航天制造公司'}


# 属性

In [5]:
property(input("Node:"))

Node:肉
ChineseName : 《肉》
ForeignNames : flesh;meat;KangXiradical
category : 剧情
pinyin : ròu
TheDirector : 保罗·莫里西
TheWriters : 保罗·莫里西
starring : 乔·达拉斯安铎、杰拉尔丁·史密斯
PhoneticNotation : ㄖㄡˋ
structure : 单一结构
radical : 肉
radical : 肉
stroke : 6
stroke : 6
ReleaseArea : 美国
LanguageVersion : 普通话
LanguageVersion : 普通话
ThePictureColor : 彩色
ThePictureColor : 彩色
AmbiguousRelationship : 肉[谢天笑演唱歌曲]
AmbiguousRelationship : 肉[MC半阳原创歌曲]
AmbiguousRelationship : 肉[刘润洁演唱歌曲]
AmbiguousRelationship : 肉[肉类总称]
AmbiguousRelationship : 肉[汉字]
ChineseName : 《肉》
ForeignNames : flesh;meat;KangXiradical
category : 剧情
pinyin : ròu
TheDirector : 保罗·莫里西
TheWriters : 保罗·莫里西
starring : 乔·达拉斯安铎、杰拉尔丁·史密斯
PhoneticNotation : ㄖㄡˋ
structure : 单一结构
radical : 肉
radical : 肉
stroke : 6
stroke : 6
ReleaseArea : 美国
LanguageVersion : 普通话
LanguageVersion : 普通话
ThePictureColor : 彩色
ThePictureColor : 彩色
AmbiguousRelationship : 肉[谢天笑演唱歌曲]
AmbiguousRelationship : 肉[MC半阳原创歌曲]
AmbiguousRelationship : 肉[刘润洁演唱歌曲]
AmbiguousRelationship : 肉[肉类总称]
A

# 推理

In [9]:
QA_test()

                                        Q:宇智波鼬的身高？
----------------------------------------------------------------------------------------------------
A: 175.2cm→178cm
                                        Q:须佐能乎·十拳剑的使用者
----------------------------------------------------------------------------------------------------
A: 鼬
                                        Q:多糖体是什么
----------------------------------------------------------------------------------------------------
A: 多糖体是一种长链糖，多种菇类及其菌丝都富含多糖体，对人体免疫系统极其有益。
                                        Q:周杰伦住哪
----------------------------------------------------------------------------------------------------
A: 周杰伦（JayChou），1979年1月18日出生于台湾省新北市，中国台湾流行乐男歌手、音乐人、演员、导演、编剧、监制、商人。
                                        Q:周杰伦是谁
----------------------------------------------------------------------------------------------------
A: 周杰伦（JayChou），1979年1月18日出生于台湾省新北市，中国台湾流行乐男歌手、音乐人、演员、导演、编剧、监制、商人。
                                        Q:周杰伦的血